# IKT457 - Assignment 2



In [133]:
import random
from random import choice

#### Data: 6 Patients (3 Recurrence, 3 Non-recurrence)

In [134]:
# Recurrence (cancer) patients
cancer_patients = [
    {"lt40": False, "ge40": True,  "premono": False,
     "zero_two": False, "three_five": True, "six_eight": False,
     "deg_malig_1": False, "deg_malig_2": False, "deg_malig_3": True},

    {"lt40": False, "ge40": True,  "premono": False,
     "zero_two": False, "three_five": False, "six_eight": True,
     "deg_malig_1": False, "deg_malig_2": False, "deg_malig_3": True},

    {"lt40": False, "ge40": False, "premono": True,
     "zero_two": True, "three_five": False, "six_eight": False,
     "deg_malig_1": False, "deg_malig_2": False, "deg_malig_3": True},
]

# Non-recurrence (cancer-free) patients
cancer_free_patients = [
    {"lt40": True,  "ge40": False, "premono": False,
     "zero_two": True, "three_five": False, "six_eight": False,
     "deg_malig_1": False, "deg_malig_2": False, "deg_malig_3": True},

    {"lt40": False, "ge40": True,  "premono": False,
     "zero_two": True, "three_five": False, "six_eight": False,
     "deg_malig_1": False, "deg_malig_2": True,  "deg_malig_3": False},

    {"lt40": False, "ge40": False, "premono": True,
     "zero_two": True, "three_five": False, "six_eight": False,
     "deg_malig_1": True,  "deg_malig_2": False, "deg_malig_3": False},
]

#### Rule Evaluation

A _condition_ is a list of literals like `["deg_malig_3", "NOT lt40"]`. \
The `evaluate_condition` function checks whether an observation satisfies all literals.

In [135]:
def evaluate_condition(observation, condition):
    truth_value_of_condition = True
    for feature in observation:
        if feature in condition and observation[feature] == False:
            truth_value_of_condition = False
            break
        if 'NOT ' + feature in condition and observation[feature] == True:
            truth_value_of_condition = False
            break
    return truth_value_of_condition

#### Rules R1, R2, R3 (from slides)

* R1 **if** _Deg-malign 3_ **and not** _Menopause lt40_ **then** _Recurrence_, 
* R2 **if** _Deg-malign 3_ **and not** _Menopause lt40_ **then** _Recurrence_, 
* R3 **if** _Inv-nodes 0-2_ **then** Non-Recurrence. 


In [136]:
# Rules
R1 = ["deg_malig_3", "NOT lt40"]  # -> Recurrence
R2 = ["deg_malig_3", "NOT lt40"]  # -> Recurrence
R3 = ["zero_two"]  # -> Non-recurrence

n = 1
print(f"\n###### Rule 1 ######")
for i in range(len(cancer_patients)):
    print(f"Patient {n}, {evaluate_condition(cancer_patients[i], R1)}")
    n += 1 
    print(f"Patient {n}, {evaluate_condition(cancer_free_patients[i], R1)}")
    n += 1
print(f"____________________________________________")

n = 1
print(f"\n###### Rule 2 ######")
for i in range(len(cancer_patients)):
    print(f"Patient {n}, {evaluate_condition(cancer_patients[i], R2)}")
    n += 1 
    print(f"Patient {n}, {evaluate_condition(cancer_free_patients[i], R2)}")
    n += 1
print(f"____________________________________________")

n = 1
print(f"\n###### Rule 3 ######")
for i in range(len(cancer_patients)):
    print(f"Patient {n}, {evaluate_condition(cancer_patients[i], R3)}")
    n += 1 
    print(f"Patient {n}, {evaluate_condition(cancer_free_patients[i], R3)}")
    n += 1
print(f"____________________________________________\n")


###### Rule 1 ######
Patient 1, True
Patient 2, False
Patient 3, True
Patient 4, False
Patient 5, True
Patient 6, False
____________________________________________

###### Rule 2 ######
Patient 1, True
Patient 2, False
Patient 3, True
Patient 4, False
Patient 5, True
Patient 6, False
____________________________________________

###### Rule 3 ######
Patient 1, False
Patient 2, True
Patient 3, False
Patient 4, True
Patient 5, True
Patient 6, True
____________________________________________



### Task 1: Classify all 6 patients using R1, R2, R3

We do a voting. Each satisifed recurrence rule gives +1 vote, each satisfied non-recurrence rule gives -1. 

In [137]:
def classify_with_manual_rules(observation, R1, R2, R3):
    vote_sum = 0
    if evaluate_condition(observation, R1):
        vote_sum += 1
    if evaluate_condition(observation, R2):
        vote_sum += 1
    if evaluate_condition(observation, R3):
        vote_sum -= 1

    label = "Cancer" if vote_sum >= 0 else "Cancer free"
    return vote_sum, label


patient_id = 1
for i in range(len(cancer_patients)):
    vote, result = classify_with_manual_rules(cancer_patients[i], R1, R2, R3)
    print(f"Patient {patient_id}: Vote = {vote} -> {result}")
    patient_id += 1

    vote, result = classify_with_manual_rules(cancer_free_patients[i], R1, R2, R3)
    print(f"Patient {patient_id}: Vote = {vote} -> {result}")
    patient_id += 1

Patient 1: Vote = 2 -> Cancer
Patient 2: Vote = -1 -> Cancer free
Patient 3: Vote = 2 -> Cancer
Patient 4: Vote = -1 -> Cancer free
Patient 5: Vote = 1 -> Cancer
Patient 6: Vote = -1 -> Cancer free


In [138]:
class Memory:
    def __init__(self, forget_value, memorize_value, memory):
        self.memory = memory
        self.forget_value = forget_value
        self.memorize_value = memorize_value

    def get_memory(self):
        return self.memory
    
    def get_literals(self):
        return list(self.memory.keys())
    
    def get_condition(self):
        condition = []
        for literal in self.memory:
            if self.memory[literal] >= 6:
                condition.append(literal)
        return condition
    
    def memorize(self, literal):
        if random.random() <= self.memorize_value and self.memory[literal] < 10:
            self.memory[literal] += 1

    def forget(self, literal):
        if random.random() <= self.forget_value and self.memory[literal] > 1:
            self.memory[literal] -= 1

    def memorize_always(self, literal):
        if self.memory[literal] < 10:
            self.memory[literal] += 1

**Type I feedback:** Reinforces literals that match the current observation (and penalizes others with forgetting).  
**Type II feedback:** If the current condition fires on a negative example, it strengthens negations to suppress it.

In [139]:
def type_i_feedback(observation, memory):
    remaining_literals = memory.get_literals()
    if evaluate_condition(observation, memory.get_condition()) == True:
        for feature in observation:
            if observation[feature] == True:
                memory.memorize(feature)
                remaining_literals.remove(feature)
            elif observation[feature] == False:
                memory.memorize("NOT " + feature)
                remaining_literals.remove("NOT " + feature)
    for literal in remaining_literals:
        memory.forget(literal)

def type_ii_feedback(observation, memory):
    if evaluate_condition(observation, memory.get_condition()) == True:
        for feature in observation:
            if observation[feature] == False:
                memory.memorize_always(feature)
            elif observation[feature] == True:
                memory.memorize_always("NOT " + feature)



#### Training Utilities

We learn one cluse for Recurrence and one clause for Non-recurrence by mixing Type I and Type II feedback.

In [140]:
def classify_patient(observation, pos_rules, neg_rules):
    vote_sum = 0
    for pos_rule in pos_rules:
        if evaluate_condition(observation, pos_rule.get_condition()) == True:
            vote_sum += 1
    for neg_rule in neg_rules:
        if evaluate_condition(observation, neg_rule.get_condition()) == True:
            vote_sum -= 1

    if vote_sum >= 0:
        return "Cancer"         # Recurrence
    else:
        return "Cancer free"    # Non-recurrence
    

def train(cancer_patients, cancer_free_patients, cancer_rule, cancer_free_rule, iterations=100):
    # Learn recurrence clause
    for _ in range(iterations):
        observation_id = choice([0, 1, 2])
        cancer = choice([0, 1])
        if cancer == 1:
            type_i_feedback(cancer_patients[observation_id], cancer_rule)
        else:
            type_ii_feedback(cancer_free_patients[observation_id], cancer_rule)

    # Learn non-recurrence clause
    for _ in range(iterations):
        observation_id = choice([0, 1, 2])
        cancer = choice([0, 1])
        if cancer == 1:
            type_i_feedback(cancer_free_patients[observation_id], cancer_free_rule)
        else:
            type_ii_feedback(cancer_patients[observation_id], cancer_free_rule)

### Task 2 & 3: Experiments

Run the learning with three forget and memorize setting and see the learned conditions and accuracy.

#### Forget value 0.8 and memorize value 0.2


In [141]:
cancer_mem = {
    "lt40": 5, "ge40": 5, "premono": 5, 
    "zero_two": 5, "three_five": 5, "six_eight": 5, 
    "deg_malig_1": 5, "deg_malig_2": 5, "deg_malig_3": 5,

    "NOT lt40": 5, "NOT ge40": 5, "NOT premono": 5, 
    "NOT zero_two": 5, "NOT three_five": 5, "NOT six_eight": 5, 
    "NOT deg_malig_1": 5, "NOT deg_malig_2": 5, "NOT deg_malig_3": 5,
}

cancer_free_mem = {
    "lt40": 5, "ge40": 5, "premono": 5, 
    "zero_two": 5, "three_five": 5, "six_eight": 5, 
    "deg_malig_1": 5, "deg_malig_2": 5, "deg_malig_3": 5,

    "NOT lt40": 5, "NOT ge40": 5, "NOT premono": 5, 
    "NOT zero_two": 5, "NOT three_five": 5, "NOT six_eight": 5, 
    "NOT deg_malig_1": 5, "NOT deg_malig_2": 5, "NOT deg_malig_3": 5,
}

iter = 400
cancer_recurrence = 0
cancer_free_recurrence = 0

cancer_rule = Memory(0.8, 0.2, cancer_mem)
cancer_free_rule = Memory(0.8, 0.2, cancer_free_mem)

for _ in range(iter):

    train(cancer_patients, cancer_free_patients, cancer_rule, cancer_free_rule, iterations=100)

    for j in range(3):
        if classify_patient(cancer_patients[j], [cancer_rule], [cancer_free_rule]) == "Cancer":
            cancer_recurrence += 1
    

    for j in range(3):
        if classify_patient(cancer_free_patients[j], [cancer_rule], [cancer_free_rule]) == "Cancer free":
            cancer_free_recurrence += 1

    
print(f"Recurrence rule: {cancer_rule.get_condition()}")
print(f"Non-recurrence rule: {cancer_free_rule.get_condition()}\n")

print(f"Cancer patients: {cancer_recurrence} / {iter * 3}")
print(f"Cancer free patients: {cancer_free_recurrence} / {iter * 3}")
print(f"Total: {cancer_recurrence + cancer_free_recurrence} / {iter * 6}")


Recurrence rule: ['deg_malig_3', 'NOT lt40', 'NOT deg_malig_1', 'NOT deg_malig_2']
Non-recurrence rule: ['NOT three_five']

Cancer patients: 1188 / 1200
Cancer free patients: 915 / 1200
Total: 2103 / 2400


#### Forget value 0.5 and memorize value 0.5

In [142]:
cancer_mem = {
    "lt40": 5, "ge40": 5, "premono": 5, 
    "zero_two": 5, "three_five": 5, "six_eight": 5, 
    "deg_malig_1": 5, "deg_malig_2": 5, "deg_malig_3": 5,

    "NOT lt40": 5, "NOT ge40": 5, "NOT premono": 5, 
    "NOT zero_two": 5, "NOT three_five": 5, "NOT six_eight": 5, 
    "NOT deg_malig_1": 5, "NOT deg_malig_2": 5, "NOT deg_malig_3": 5,
}

cancer_free_mem = {
    "lt40": 5, "ge40": 5, "premono": 5, 
    "zero_two": 5, "three_five": 5, "six_eight": 5, 
    "deg_malig_1": 5, "deg_malig_2": 5, "deg_malig_3": 5,

    "NOT lt40": 5, "NOT ge40": 5, "NOT premono": 5, 
    "NOT zero_two": 5, "NOT three_five": 5, "NOT six_eight": 5, 
    "NOT deg_malig_1": 5, "NOT deg_malig_2": 5, "NOT deg_malig_3": 5,
}

iter = 400
cancer_recurrence = 0
cancer_free_recurrence = 0

cancer_rule = Memory(0.5, 0.5, cancer_mem)
cancer_free_rule = Memory(0.5, 0.5, cancer_free_mem)

for _ in range(iter):

    train(cancer_patients, cancer_free_patients, cancer_rule, cancer_free_rule, iterations=100)

    for j in range(3):
        if classify_patient(cancer_patients[j], [cancer_rule], [cancer_free_rule]) == "Cancer":
            cancer_recurrence += 1
    

    for j in range(3):
        if classify_patient(cancer_free_patients[j], [cancer_rule], [cancer_free_rule]) == "Cancer free":
            cancer_free_recurrence += 1


print(f"Recurrence rule: {cancer_rule.get_condition()}")
print(f"Non-recurrence rule: {cancer_free_rule.get_condition()}\n")

print(f"Cancer patients: {cancer_recurrence} / {iter * 3}")
print(f"Cancer free patients: {cancer_free_recurrence} / {iter * 3}")
print(f"Total: {cancer_recurrence + cancer_free_recurrence} / {iter * 6}")

Recurrence rule: ['ge40', 'deg_malig_3', 'NOT lt40', 'NOT premono', 'NOT zero_two', 'NOT deg_malig_1', 'NOT deg_malig_2']
Non-recurrence rule: ['zero_two', 'NOT six_eight']

Cancer patients: 1187 / 1200
Cancer free patients: 723 / 1200
Total: 1910 / 2400


#### Forget value 0.2 and memorize value 0.8

In [143]:
cancer_mem = {
    "lt40": 5, "ge40": 5, "premono": 5, 
    "zero_two": 5, "three_five": 5, "six_eight": 5, 
    "deg_malig_1": 5, "deg_malig_2": 5, "deg_malig_3": 5,

    "NOT lt40": 5, "NOT ge40": 5, "NOT premono": 5, 
    "NOT zero_two": 5, "NOT three_five": 5, "NOT six_eight": 5, 
    "NOT deg_malig_1": 5, "NOT deg_malig_2": 5, "NOT deg_malig_3": 5,
}

cancer_free_mem = {
    "lt40": 5, "ge40": 5, "premono": 5, 
    "zero_two": 5, "three_five": 5, "six_eight": 5, 
    "deg_malig_1": 5, "deg_malig_2": 5, "deg_malig_3": 5,

    "NOT lt40": 5, "NOT ge40": 5, "NOT premono": 5, 
    "NOT zero_two": 5, "NOT three_five": 5, "NOT six_eight": 5, 
    "NOT deg_malig_1": 5, "NOT deg_malig_2": 5, "NOT deg_malig_3": 5,
}

iter = 400
cancer_recurrence = 0
cancer_free_recurrence = 0

cancer_rule = Memory(0.2, 0.8, cancer_mem)
cancer_free_rule = Memory(0.2, 0.8, cancer_free_mem)

for _ in range(iter):

    train(cancer_patients, cancer_free_patients, cancer_rule, cancer_free_rule, iterations=100)

    for j in range(3):
        if classify_patient(cancer_patients[j], [cancer_rule], [cancer_free_rule]) == "Cancer":
            cancer_recurrence += 1
    

    for j in range(3):
        if classify_patient(cancer_free_patients[j], [cancer_rule], [cancer_free_rule]) == "Cancer free":
            cancer_free_recurrence += 1


print(f"Recurrence rule: {cancer_rule.get_condition()}")
print(f"Non-recurrence rule: {cancer_free_rule.get_condition()}\n")

print(f"Cancer patients: {cancer_recurrence} / {iter * 3}")
print(f"Cancer free patients: {cancer_free_recurrence} / {iter * 3}")
print(f"Total: {cancer_recurrence + cancer_free_recurrence} / {iter * 6}")

Recurrence rule: ['ge40', 'three_five', 'deg_malig_3', 'NOT lt40', 'NOT premono', 'NOT zero_two', 'NOT six_eight', 'NOT deg_malig_1', 'NOT deg_malig_2']
Non-recurrence rule: ['lt40', 'zero_two', 'NOT ge40', 'NOT premono', 'NOT three_five', 'NOT six_eight', 'NOT deg_malig_1', 'NOT deg_malig_2']

Cancer patients: 1200 / 1200
Cancer free patients: 402 / 1200
Total: 1602 / 2400


## Discussion

- When the **memorize** value is high compared to **forget**, the rules keep more literals.  
  This can make the rules too detailed for such a small dataset.  
- When the **forget** value is high, the rules become shorter because more literals are removed.  
- Since we only use one rule for each class and the dataset is very small (3 + 3 patients),  
  the results can change a bit each time we run the training.  

